In [1]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [10]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.LocalSession() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

# Upload data for training

In [ ]:
! ./upload_coco2017_to_s3.sh "sagemaker-us-east-2-553020858742" "detectron2/datasets/coco"

Create stage directory: /home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41
--2020-03-31 18:56:41--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.184.211
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.184.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘/home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip’

/home/ec2-user/Sage 100%[===================>]  18.01G  36.3MB/s    in 9m 46s  

2020-03-31 19:06:27 (31.5 MB/s) - ‘/home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip’ saved [19336861798/19336861798]

./upload_coco2017_to_s3.sh: line 19: zips/train2017.zip: No such file or directory
Extracting /home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip

# Train your model

In [16]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = "us-east-2"
container = "d2-sm-coco"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, container)

hyperparameters = {"config-file":"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"}

d2 = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=1, 
#                                   train_instance_type='ml.p3.2xlarge',
                                   train_instance_type='local_gpu',
#                                   train_volume_size=100, TODO: uncomment after debugging
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   hyperparameters = hyperparameters,
                                   sagemaker_session=sess)

# TODO: debugging scripts without data
#d2.fit({'train':"s3://sagemaker-us-east-2-553020858742/balloon/train",
#        'val':"s3://sagemaker-us-east-2-553020858742/balloon/val"}) 

d2.fit()


Creating tmp_rl8yfa9_algo-1-aivw1_1 ... 
Attaching to tmp_rl8yfa9_algo-1-aivw1_12mdone
algo-1-aivw1_1  | 2020-04-08 12:21:44,178 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-aivw1_1  | 2020-04-08 12:21:44,179 sagemaker-containers INFO     Failed to parse hyperparameter config-file value COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml to Json.
algo-1-aivw1_1  | Returning the value itself
algo-1-aivw1_1  | 2020-04-08 12:21:44,257 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-aivw1_1  | 2020-04-08 12:21:44,260 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-aivw1_1  | 2020-04-08 12:21:44,330 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
algo-1-aivw1_1  | Generating setup.py
algo-1-aivw1_1  | 2020-04-08 12:21:44,330 sagemaker-containers INFO     Generating setup.cfg
algo-1-aivw1_1  | 2020-04-08 12:21:44,331 s

Failed to delete: /tmp/tmp_rl8yfa9/algo-1-aivw1 Please remove it manually.


tmp_rl8yfa9_algo-1-aivw1_1 exited with code 1
Aborting on container exit...


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp_rl8yfa9/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1